# Choosing a U.S. City to Live In To Pursue Data Science Career

## Introduction 

Pursuing a Data Science career is a exciting option for scientists and business people wanting to break into the technology industry.

For people transitioning to a data science career, a consideration is whether or not it would be beneficial to move to some of the cities known as tech hubs such as San Francisco. Although this is an attractive option, there are other considerations that should be weighed in order to make the best decision.

Apart from career prospects, one should choose a new home based on personal preferences. 

In this project, I will gather some data about several cities in mind and determine which one is the best fit for my preferences and my new career.

## Objective

To evaluate several U.S. cities more objectively based on personal preferences for lifestyle and health. 

## Preferences

- Weather: A mild weather is preferred.
- Pollen and mold: Lower pollen and mold counts are preferred.
- Scenery: A city near mountains is preferred.
- Urbanization and beautification: A city with a large number of parks is preferred.
- Outdoors: the availability of hiking trails and outdoor venues is preferred.
- Political views: blue state
- Career: tech hub.

## Audience

This project may be of interest to any person trying to figure out where to move. In order to make an objective, responsible decision, one must research and weigh pros and cons.

Machine Learning may be better able to determine where we should move than ourselves.

## Data

I will gather data from various U.S. Cities that will be correlated to the measures listed above. For example, the number of recycling centers may be correlated with political affiliation.
Presence of tech startups and coworking spaces is a proxy for tech hub.
Spiritual centers would be a measure of diversity of thought.
Street Art, Sculpture, Botanical gardens, parks, trees, and trails will be proxy of beautification.
University will be proxy of educational status of population.

# Methodology

Firstly, Foursquare data 

## Libraries

In [1]:

import pandas as pd
import numpy as np
import requests

import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors

import plotly
import plotly.plotly as py
import plotly.figure_factory as ff


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from pandas.io.html import read_html
from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.91 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  37.00 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  46.28 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.13 MB/s
vincent-0.4.4- 100% |###################

In [108]:
import matplotlib.pyplot as plt

# Methodology

### 1. Create a list with cities of interest

In [109]:
# Enter the names of the cities of interest
cities = ['Houston, TX','Austin, TX','Denver, CO','Seattle, WA','San Francisco, CA','Portland, OR']

### 2. Look up in Foursquare Docs the venue category IDs that match personal preferences and store them in thematical lists

In [113]:
#Outdoors and Recreation Venues: Trails, Bike Trail, Botanical Gardens, Forest, Mountain, Nature Preserve, National Park, Park, Tree, Outdoor Event Space

outdoors_venues_ID = ['4bf58dd8d48988d159941735','56aa371be4b08b9a8d57355e','52e81612bcbc57f1066b7a22','52e81612bcbc57f1066b7a23','4eb1d4d54b900d56c88a45fc','52e81612bcbc57f1066b7a13','52e81612bcbc57f1066b7a21','4bf58dd8d48988d163941735','56aa371be4b08b9a8d57356a']
                      
# Professional & Other Places:  Tech Startup

professional_venues_ID = ['4bf58dd8d48988d125941735']
    
#cultural venues:  Spiritual Center: Buddhist Temple, Hindu Temple, 

cultural_venues_ID = ['52e81612bcbc57f1066b7a3e','52e81612bcbc57f1066b7a3f']

# Food and drink shop: Farmers Market, Health Food Store, Organic Grocery, Fruit and Vegetable Store, Juice Bar

food_venues_ID = ['4bf58dd8d48988d1fa941735','50aa9e744b90af0d42d5de0e','52f2ab2ebcbc57f1066b8b45','52f2ab2ebcbc57f1066b8b1c','4bf58dd8d48988d112941735']

# Beautification: Park, 

beauty_venues_ID = ['4bf58dd8d48988d163941735']

categoryIDs = [outdoors_venues_ID,professional_venues_ID,cultural_venues_ID,food_venues_ID,beauty_venues_ID]

# create a dataframe to extract the Foursquare data into.



### 3. Create a Function to Connect and Extract Data from Foursquare

In [120]:
# This function connects to Foursquare and extracts venues matching a CategoryID and 
# stores them in the dataframe designated.

def getFoursquareCityData(cities, categoryIDs, limit, max_radius):

    # Connect to Foursquare and Query each city to find the number of each venue.

    client_ID = 'HJQTB2PO3CQ31PY0D3MKAFCODL1XOO2RLY3VXWZ2XVOUHERI'
    client_secret = 'YVXU2GICCUXXHV00HDZUG2ZCR5WG50VYWQCCF14A5JJYY31Y'
    version = '20180605' # Foursquare API version

    print('Your credentails:')
    print('CLIENT_ID: ' + client_ID)
    print('CLIENT_SECRET:' + client_secret)

    venues_list = []
    venues_df = pd.DataFrame(columns = ['City','CategoryID','Venue','Latitude','Longitude','Type'])
    
    for city in cities:
        for list in categoryIDs:
            for category in list:
                url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'.format(
                    client_ID,
                    client_secret,
                    version,
                    city,
                    max_radius,
                    limit,
                    category)

                city_abr = city.upper()[:3]
                try:
                    venues = requests.get(url).json()['response']['groups'][0]['items']

                    venues_list.append([(
                    city,
                    category,
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],
                    v['venue']['categories'][0]['name']) for v in venues])
                except IndexError:
                    continue
                except KeyError:
                    continue

            venues_df = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        print(city_abr + " records extracted.")
    print("The size of your venue dataframe is:")
    print(venues_df.shape)
    return venues_df
 

### 4. Populate the venues dataframe with the raw data

In [122]:
venues_df = getFoursquareCityData(cities, categoryIDs, 100, 100000)

Your credentails:
CLIENT_ID: HJQTB2PO3CQ31PY0D3MKAFCODL1XOO2RLY3VXWZ2XVOUHERI
CLIENT_SECRET:YVXU2GICCUXXHV00HDZUG2ZCR5WG50VYWQCCF14A5JJYY31Y
HOU records extracted.
AUS records extracted.
DEN records extracted.
SEA records extracted.
SAN records extracted.
POR records extracted.
The size of your venue dataframe is:
(5964, 6)


In [129]:
venues_df.columns = ['City','CategoryID','Venue','Latitude','Longitude','Type']

### 5. QC and Filter the Foursquare data in the dataframe

#### Data QC

In [143]:
venues_df.head()

,City,CategoryID,Venue,Latitude,Longitude,Type
0,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Walk,29.762177,-95.375844,Trail
1,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Park,29.762068,-95.391626,Park
2,"Houston, TX",4bf58dd8d48988d159941735,Houston Arboretum & Nature Center,29.765361,-95.452177,Botanical Garden
3,"Houston, TX",4bf58dd8d48988d159941735,Herman Park Trails,29.719804,-95.388748,Trail
4,"Houston, TX",4bf58dd8d48988d159941735,Terry Hershey Park,29.779138,-95.623096,Park


First observations: 
- Found out Tree in Foursquare corresponds to tree cutting services - deleted it from categories for final report
- Found out duplicates are present
- Apparently there is a Hogwarts campus in Austin???
- University, Library, Coworking Space as parameters are not as useful as initially we though. Deleted from categories.

Drop Duplicates!

In [144]:
venues_df = venues_df.drop_duplicates()

In [145]:
venues_df.shape

(5364, 6)

Deleted that there were 600 duplicate records!

#### Filtering the Data

Let's find out which city has the most venues of interest and which one the least.

In [146]:
# This block of code creates a new dataframe that counts how many total venues are found per city
venues_count = pd.DataFrame(venues_df.groupby('City').count()['Venue'])
venues_count = venues_count.reset_index()
max_number = venues_count['Venue'].max()
max_city = venues_count.iloc[venues_count['Venue'].idxmax()][0]
min_number = venues_count['Venue'].min()
min_city = venues_count.iloc[venues_count['Venue'].idxmin()][0]
print('The city with the highest amount of venues matching your interests is: ' + str(max_city) +
      ' with ' + str(max_number) + ' venues.')
print('The city with the lowest amount of venues matching your interests is: ' + str(min_city) +
      ' with ' + str(min_number) + ' venues.')

The city with the highest amount of venues matching your interests is: San Francisco, CA with 1258 venues.
The city with the lowest amount of venues matching your interests is: Houston, TX with 729 venues.


Let's find out which venue types are more relevant to include in our analysis

In [61]:
# create lists of cities and number of venues for easy graphical representation
N = len(cities)
cities = []
total_number_venues_list = []
for index in range(0,N):
    city_total = total_venues.iloc[index][1]
    cities.append(total_venues.iloc[index][0])
    total_number_venues_list.append(city_total)

In [147]:
# Create a dataframe that counts the total number of venues per type found.
venue_types_df = pd.DataFrame(venues_df.groupby('Type').count())

In [148]:
venue_types_df

,City,CategoryID,Venue,Latitude,Longitude
Type,,,,,
American Restaurant,3,3,3,3,3
Amphitheater,3,3,3,3,3
Athletics & Sports,2,2,2,2,2
Bakery,7,7,7,7,7
Bar,1,1,1,1,1
Beach,8,8,8,8,8
Big Box Store,1,1,1,1,1
Bike Rental / Bike Share,2,2,2,2,2
Bike Shop,5,5,5,5,5


We see here that some venues seem less significant than others, so we are going to filter out the least common types 
and create a list of the relevant types.

In [163]:
# delete types that don't meet a certain thresh

relevant_venues_df = total_venues[total_venues['City'] > 1]
relevant_venues_df = relevant_venues_df.reset_index()


In [157]:
# create a list of relevant venues

In [165]:
relevant_types = relevant_venues_df['Type'].tolist()
relevant_types


['American Restaurant',
 'Amphitheater',
 'Athletics & Sports',
 'Bakery',
 'Beach',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Bike Trail',
 'Botanical Garden',
 'Breakfast Spot',
 'Bridge',
 'Buddhist Temple',
 'Building',
 'Business Service',
 'Café',
 'Campground',
 'City',
 'Coffee Shop',
 'Coworking Space',
 'Dog Run',
 'Event Space',
 'Farm',
 'Farmers Market',
 'Field',
 'Flea Market',
 'Food & Drink Shop',
 'Food Truck',
 'Forest',
 'Fruit & Vegetable Store',
 'Garden',
 'General Entertainment',
 'General Travel',
 'Gift Shop',
 'Gourmet Shop',
 'Grocery Store',
 'Gym',
 'Gym / Fitness Center',
 'Harbor / Marina',
 'Health Food Store',
 'Heliport',
 'Hindu Temple',
 'Historic Site',
 'History Museum',
 'Juice Bar',
 'Lake',
 'Market',
 'Miscellaneous Shop',
 'Mountain',
 'National Park',
 'Nature Preserve',
 'Neighborhood',
 'Office',
 'Organic Grocery',
 'Other Great Outdoors',
 'Outdoor Event Space',
 'Park',
 'Pharmacy',
 'Playground',
 'Plaza',
 'Pool',
 'Restaurant',
 '

#### Create final filtered dataframe

In [168]:
#Filter out irrelevant types from dataset
# Create now the final dataframe where the irrelevant types are 
df = venues_df
df = df.loc[df['Type'].isin(relevant_types)]
df.shape

(5321, 6)

In [169]:
df.head()

,City,CategoryID,Venue,Latitude,Longitude,Type
0,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Walk,29.762177,-95.375844,Trail
1,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Park,29.762068,-95.391626,Park
2,"Houston, TX",4bf58dd8d48988d159941735,Houston Arboretum & Nature Center,29.765361,-95.452177,Botanical Garden
3,"Houston, TX",4bf58dd8d48988d159941735,Herman Park Trails,29.719804,-95.388748,Trail
4,"Houston, TX",4bf58dd8d48988d159941735,Terry Hershey Park,29.779138,-95.623096,Park


### 6. Plot Data

Prepare the data for plotting.
Create various slices of the data for input in a horizontal bar chart.

In [171]:
# Create dataframes for each venue category

outdoors_df = df.loc[df['CategoryID'].isin(outdoors_venues_ID)]
startups_df = df.loc[df['CategoryID'].isin(professional_venues_ID)]
cultural_df = df.loc[df['CategoryID'].isin(cultural_venues_ID)]
food_df = df.loc[df['CategoryID'].isin(food_venues_ID)]
beauty_df = df.loc[df['CategoryID'].isin(beauty_venues_ID)]

In [173]:
# Count the numbers of venues in each category.
outdoors_count = pd.DataFrame(outdoors_df.groupby('City').count()['Venue'])
startups_count = pd.DataFrame(startups_df.groupby('City').count()['Venue'])
cultural_count = pd.DataFrame(cultural_df.groupby('City').count()['Venue'])
food_count = pd.DataFrame(food_df.groupby('City').count()['Venue'])
beauty_count = pd.DataFrame(beauty_df.groupby('City').count()['Venue'])

In [174]:
# Create a function to count how many of each category are present per city

def makeList(cities, count_df):
    N = len(cities)
    count_list = []
    for index in range(0,N):
        city_total = count_df.iloc[index][0]
        count_list.append(city_total)
    return count_list

In [175]:
# Apply the function to create lists
outdoors_count_list = makeList(cities, outdoors_count)
startups_count_list = makeList(cities, startups_count)
cultural_count_list = makeList(cities, cultural_count)
food_count_list = makeList(cities, food_count)
beauty_count_list = makeList(cities, beauty_count)


['Houston, TX',
 'Austin, TX',
 'Denver, CO',
 'Seattle, WA',
 'San Francisco, CA',
 'Portland, OR']

In [177]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    y= cities,
    x= outdoors_count_list,
    name='Outdoors',
    orientation = 'h',
    marker = dict(
        color = 'rgba(73,124,81,0.8)',
    )
)
trace2 = go.Bar(
    y= cities,
    x= startups_count_list,
    name='Startups',
    orientation = 'h',
    marker = dict(
        color = 'rgba(81,73,124,0.8)'
   )
)
trace3 = go.Bar(
    y= cities,
    x= cultural_count_list,
    name='Cultural',
    orientation = 'h',
    marker = dict(
        color = 'rgba(124,73,116,0.8)',
   )
)
trace4 = go.Bar(
    y= cities,
    x= food_count_list,
    name='Food',
    orientation = 'h',
    marker = dict(
        color = 'rgba(139,115,95,0.8)',
    )
)
trace5 = go.Bar(
    y= cities,
    x= beauty_count_list,
    name='Beauty',
    orientation = 'h',
    marker = dict(
        color = 'rgba(95,119,139,0.8)',
    )
)

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

We observe that there are more startups than the limit imposed by Foursquare. WE could add more points inside the city, or we can rely on data points that are more predicitive.
According to this analysis, Houston and Denver look very similar, but we know this is not the case.

### Source 2: Weather Data

In [212]:
outdoors_df

,City,CategoryID,Venue,Latitude,Longitude,Type,Outdoors Venue
0,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Walk,29.762177,-95.375844,Trail,0
1,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Park,29.762068,-95.391626,Park,0
2,"Houston, TX",4bf58dd8d48988d159941735,Houston Arboretum & Nature Center,29.765361,-95.452177,Botanical Garden,0
3,"Houston, TX",4bf58dd8d48988d159941735,Herman Park Trails,29.719804,-95.388748,Trail,0
4,"Houston, TX",4bf58dd8d48988d159941735,Terry Hershey Park,29.779138,-95.623096,Park,0
5,"Houston, TX",4bf58dd8d48988d159941735,Ho Chi Minh: Memorial Park Mountain Bike Trails,29.765167,-95.444738,Other Great Outdoors,0
6,"Houston, TX",4bf58dd8d48988d159941735,Buffalo Bayou Loop,29.761345,-95.401556,Trail,0
7,"Houston, TX",4bf58dd8d48988d159941735,Memorial/Allen Parkway Trails,29.760212,-95.408612,Trail,0
8,"Houston, TX",4bf58dd8d48988d159941735,Memorial Park,29.767656,-95.442524,Park,0
9,"Houston, TX",4bf58dd8d48988d159941735,Eleanor Tinsley Park,29.761440,-95.379271,Trail,0


(247, 6)

In [ ]:
relevant_venues_df = venues_df

In [174]:
len(total_venues)

121

In [9]:
index

NameError: name 'index' is not defined

We need to find weather data on the cities of interest. This would include temperature data & precipitation.

Houston Code: USW00012918

MAX Temperature Houston 2000 - 2019 August
https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-tmax-1-8-2000-2019.csv

MIN Temperature Houston 2000 - 2019 August

https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-tmin-1-8-2000-2019.csv

AVG Temperature Houston 2000 - 2019 August

https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-tavg-1-8-2000-2019.csv

MAX Temperature Houston 2000 - 2019 Feb
https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-tmax-1-2-2000-2019.csv
MIN Temperature Houston 2000 - 2019 August Feb
https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-tmin-1-2-2000-2019.csv
AVG Temperature Houston 2000 - 2019 Feb
https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-tavg-1-2-2000-2019.csv

Annual Precipitation Houston 2000 - 2019 
https://www.ncdc.noaa.gov/cag/city/time-series/USW00012918-pcp-12-12-2000-2019.csv

AUSTIN DATA
Austin Code: USW00013958

DENVER DATA
Denver Code: USW00093037

PORTLAND DATA
Portland Code: USW00024229

SAN FRANCISCO DATA
San Frnacisco Code: USW00023234

SEATTLLE
Seattle Code: USW00024233

## Source 3: Pollen and Mold Data

We need to find weather data on the cities of interest. This would include temperature data, precipitation, humidity index, pollen count, mold spore count.


HOUSTON DATA - Station 188
http://pollen.aaaai.org/nab/index.cfm?p=AllergenCalendar&stationid=188&qsFullDate=10/1/2018

AUSTIN DATA - Station 111 

DENVER DATA - Station 196

SAN JOSE DATA - Station 108

SEATTLE DATA - Station 3

PORTLAND DATA - Station 1


Mold Spore Count Houston
http://www.houstontx.gov/health/Pollen-Mold/mold-archives.html

What the Numbers Mean
http://www.houstontx.gov/health/Pollen-Mold/numbers.html




###          Other Sources and Statistics to Consider:

I will have to think more deeply about where to find reliable data regarding these statistics and how to integrate them into my analysis: Healthiest US Cities, Best standard of living, cost of living, demographics.

This website contains open government data.
https://cities.data.gov/

## Methodology

Will use one hot encoding and a grading algorithm to find out the best city for me to live in.

## Results

## Discussion 

## Conclusion